In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv


from keras.layers import Input, Dense
from keras.models import Model
from numpy import linalg as LA
import sklearn
from sklearn import metrics
from scipy.signal import find_peaks
from numpy import asarray
from numpy import savetxt

Using TensorFlow backend.


In [4]:

def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l


usr_dir="Data_files/"
for user in range(1,23,1):
    data=[]
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    
    
    happy_bucket=[]
    sad_bucket=[]
    stressed_bucket=[]
    relax_bucket=[]
    
    happy_list=[]
    sad_list=[]
    stressed_list=[]
    relax_list=[]
    
    window_size=50
    
    print(usr_file)
    dataset=pd.read_csv(usr_file,header=None)
    dataframe=dataset.values
    
    #print("row_number="+str(dataframe.shape[0]))
    
    # value list holds unique id 
    value_list=[]
    for i in range(dataframe.shape[0]):
        value_list.append(dataframe[i][0])
        
    ID_list=list(set(value_list))
    ID_list.sort()
    print("number of unique ID="+str(len(ID_list)))
    start=0
    next_start=0
    # traverse the whole list and add list of ITD values in emotion list
    for i in range(len(ID_list)):
        ID_number=ID_list[i]
        #print("ID_number="+str(ID_number))
        ITD_values=[]
        
        for j in range(start,dataframe.shape[0],1):
            if(dataframe[j][0]==ID_number):
                ITD_values.append(dataframe[j][2])
                next_start=next_start+1
            else:
                start=next_start
                #print(next_start)
                break
        if(dataframe[j-1][4]==2 and len(ITD_values)>3):
             happy_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==-2 and len(ITD_values)>3):
             sad_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==1 and len(ITD_values)>3):
             stressed_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==0 and len(ITD_values)>3):
            relax_bucket.append(list(ITD_values))
    print("sum of bucket length="+str(len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)))
    count_ITD_list=len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)
    ########################################################
    
    for i in range(len(happy_bucket)):
        #print("happy_bucket="+str(i)+","+"length="+str(len(happy_bucket[i])))
        for j in range(0,len(happy_bucket[i]),window_size):
            if(j+window_size<len(happy_bucket[i])):
                happy_list.append(list(happy_bucket[i][j:j+window_size]))
                #print(happy_bucket[i][j:j+window_size])
                
            #else:
                #number_zero=(j+window_size)-len(happy_bucket[i])
                #happy_list.append(list(pad(happy_bucket[i][j:len(happy_bucket[i])],0,number_zero)))
                #print(pad(happy_bucket[i][j:len(happy_bucket[i])],0,number_zero))
                
    for i in range(len(sad_bucket)):
        #print("sad_bucket="+str(i)+","+"length="+str(len(sad_bucket[i])))
        for j in range(0,len(sad_bucket[i]),window_size):
            if(j+window_size<len(sad_bucket[i])):
                sad_list.append(list(sad_bucket[i][j:j+window_size]))
                #print(sad_bucket[i][j:j+window_size])   
                
    for i in range(len(stressed_bucket)):
        #print("stress_bucket="+str(i)+","+"length="+str(len(stressed_bucket[i])))
        for j in range(0,len(stressed_bucket[i]),window_size):
            if(j+window_size<len(stressed_bucket[i])):
                stressed_list.append(list(stressed_bucket[i][j:j+window_size]))
                #print(stressed_bucket[i][j:j+window_size])            
    
    for i in range(len(relax_bucket)):
        #print("relax_bucket="+str(i)+","+"length="+str(len(relax_bucket[i])))
        for j in range(0,len(relax_bucket[i]),window_size):
            if(j+window_size<len(relax_bucket[i])):
                relax_list.append(list(relax_bucket[i][j:j+window_size]))
                #print(relax_bucket[i][j:j+window_size])    
                
    for i in range(len(happy_list)):
        # peak amplitude list
        peak_list=[]
        freq_list=[]
        gap_list=[]
        # signal
        signal=np.array(happy_list[i],dtype=float)
        # taking absolute value after fft
        sp=np.abs(np.fft.fft(signal))
        # frequency from fft
        freq=np.fft.fftfreq(sp.size)
        # indicies for peaks
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
            
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        
        # find indices of top most amplitude in sorted order
        ind=peak_list.argsort()[-3:][::-1]
        #print(peak_list[ind])
        print(freq_list[ind])
        # concatenate peak amplitude and frequency
        print(peak_list[ind])
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()
        
        print(gap_list)
        
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            gap_list.remove(gap_list[-1])
        #gap_list.append(2)
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(2)    
        data.append(temp)
        # whole data is collected in data
        
        #print("peak list length in Happy="+str(peak_list[ind].size))
        #for k in range(peak_list[ind].size):
            #index=sp[peak_list[k]]
        
        
    for i in range(len(sad_list)):
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(sad_list[i],dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            gap_list.remove(gap_list[-1])
        #gap_list.append(-2)
        #print(peak_list[ind])
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(-2)    
        data.append(temp)
        
    for i in range(len(stressed_list)):
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(stressed_list[i],dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            
            gap_list.remove(gap_list[-1])
        #gap_list.append(1)
        #print(peak_list[ind])
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(1)    
        data.append(temp)
        
    for i in range(len(relax_list)):
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(relax_list[i],dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        #print(peak_list[ind])
        #print(freq[ind])
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            
            gap_list.remove(gap_list[-1])
        #gap_list.append(0)
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(0)    
        data.append(temp)  
            
    #print(data)        
    #data=asarray(data)
    data_file="Raw_data_Freq/Amp_window/"+"U"+str(user)+"_LIHF_itd_file.csv"
    with open(data_file,"w") as resultfile:
        writer=csv.writer(resultfile)
        writer.writerows(data)
        
    

Data_files/U1_LIHF_itd_file.csv
number of unique ID=114
sum of bucket length=114
[ 0.48 -0.48  0.3 ]
[11.20307292 11.20307292  6.52123368]
[15, 24, 26]
[-0.42  0.42 -0.5 ]
[8.67818479 8.67818479 7.489     ]
[21, 25, 29]
[ 0.46 -0.46 -0.42]
[7.26762292 7.26762292 6.30166948]
[23, 27, 29]
[-0.3   0.3  -0.04]
[5.36041382 5.36041382 4.42343657]
[15, 35, 48]
[ 0.46 -0.46  0.26]
[21.05997144 21.05997144 19.37825015]
[13, 23, 27]
[-0.06  0.06  0.48]
[9.22994314 9.22994314 8.49486074]
[3, 24, 47]
[ 0.18 -0.18 -0.04]
[7.03749623 7.03749623 5.94080853]
[9, 41, 48]
[-0.32  0.32  0.14]
[14.52498157 14.52498157 13.19879057]
[7, 16, 34]
[-0.48  0.48  0.44]
[11.43349736 11.43349736 10.61241392]
[22, 24, 26]
[ 0.24 -0.24 -0.1 ]
[8.1982589  8.1982589  8.08616788]
[12, 38, 45]
[-0.1   0.1   0.36]
[14.50598824 14.50598824 14.20570794]
[5, 18, 45]
[ 0.44 -0.44 -0.04]
[23.31572716 23.31572716 21.68928255]
[22, 28, 48]
[ 0.46 -0.46  0.36]
[16.65278296 16.65278296 15.67721687]
[18, 23, 27]
[-0.42  0.42 -0.5 

[19, 31, 40]
[7, 16, 34]
[21, 25, 29]
[10, 29, 40]
[20, 30, 32]
[7, 24, 26]
[8, 23, 42]
[19, 31, 35]
[19, 25, 31]
[20, 30, 44]
[18, 21, 29]
[21, 23, 27]
[19, 31, 48]
[16, 26, 34]
[15, 17, 33]
[19, 21, 29]
[21, 29, 35]
[8, 21, 29]
[20, 25, 30]
[18, 22, 32]
[20, 25, 30]
[13, 28, 37]
[4, 31, 46]
[9, 28, 41]
[17, 31, 33]
[18, 27, 32]
Data_files/U2_LIHF_itd_file.csv
number of unique ID=45
sum of bucket length=45
[ 0.42 -0.42 -0.14]
[3.73800114 3.73800114 3.33156572]
[21, 29, 43]
[-0.32  0.32 -0.38]
[5.02559064 5.02559064 4.42945684]
[16, 31, 34]
[-0.5  -0.42  0.42]
[2099.617      2093.84152472 2093.84152472]
[21, 25, 29]
[-0.4   0.4   0.06]
[11.044202   11.044202   10.67483545]
[3, 20, 30]
[ 0.34 -0.34 -0.4 ]
[9.8469574  9.8469574  9.79009005]
[17, 30, 33]
[-0.22  0.22  0.46]
[54.12336055 54.12336055 51.6132781 ]
[11, 23, 39]
[-0.5  -0.28  0.28]
[4.761      4.40362024 4.40362024]
[14, 25, 36]
[-0.16  0.16 -0.26]
[3.59216859 3.59216859 3.44736737]
[8, 37, 42]
[-0.34  0.34 -0.3 ]
[72.54310907

[5, 8, 45]
[11, 15, 39]
[14, 36, 40]
[7, 15, 43]
[3, 25, 47]
[9, 17, 41]
[15, 18, 35]
[12, 19, 38]
[19, 28, 31]
[18, 32, 36]
[16, 29, 34]
[17, 33]
[6, 22, 44]
[8, 21, 29]
[22, 25, 28]
[9, 34, 41]
[11, 19, 31]
[13, 25, 37]
[2, 18, 48]
[13, 32, 37]
[19, 31, 40]
[7, 16, 34]
[21, 25, 29]
[10, 29, 40]
[20, 30, 32]
[7, 24, 26]
[8, 23, 42]
[19, 31, 35]
[14, 36, 43]
[15, 26, 35]
[21, 25, 29]
[19, 25, 31]
[20, 30, 44]
[18, 21, 29]
[21, 23, 27]
[19, 31, 48]
[16, 26, 34]
[15, 17, 33]
[19, 21, 29]
Data_files/U4_LIHF_itd_file.csv
number of unique ID=118
sum of bucket length=118
[-0.44  0.44 -0.06]
[201.72705284 201.72705284 198.09330915]
[22, 28, 47]
[-0.5 -0.2  0.2]
[3.148      2.92616491 2.92616491]
[10, 25, 40]
[-0.26  0.26 -0.4 ]
[27.16971414 27.16971414 23.78276756]
[13, 30, 37]
[ 0.3  -0.3  -0.44]
[23.85996292 23.85996292 22.06927081]
[15, 28, 35]
[-0.24  0.24  0.48]
[42.55884833 42.55884833 40.22543706]
[12, 24, 38]
[-0.26  0.26 -0.32]
[65.39353239 65.39353239 64.57870252]
[13, 34, 37]
[-0.3

[5, 40, 45]
[23, 27, 31]
[21, 25, 29]
[10, 26, 40]
[4, 23, 27]
[20, 25, 30]
[13, 23, 37]
[11, 25, 39]
[5, 7, 45]
[3, 41, 47]
[2, 16, 34]
[10, 25, 40]
[11, 22, 28]
[21, 29, 32]
[24, 26, 48]
[11, 29, 39]
[2, 27, 48]
[8, 19, 31]
[18, 32, 36]
[7, 31, 43]
[3, 5, 47]
[10, 24, 40]
[15, 30, 35]
[12, 24, 38]
[22, 28, 39]
[23, 27, 39]
[9, 29, 41]
[19, 31, 45]
[9, 41, 45]
[14, 20, 36]
[13, 37, 48]
[19, 31, 47]
[23, 25, 27]
[2, 5, 48]
[20, 30, 34]
[4, 43, 46]
[12, 26, 38]
[14, 27, 36]
[11, 39, 43]
[16, 34, 40]
[2, 30, 48]
[12, 24, 26]
[13, 26, 37]
[17, 25, 33]
[9, 22, 28]
[16, 21, 29]
[7, 14, 43]
[13, 15, 35]
[13, 37, 40]
[11, 20, 30]
[4, 37, 46]
[9, 25, 41]
[17, 31, 33]
[24, 26, 33]
[13, 15, 35]
[11, 25, 39]
[17, 26, 33]
[11, 28, 39]
[5, 27, 45]
[7, 12, 43]
[22, 28, 45]
Data_files/U5_LIHF_itd_file.csv
number of unique ID=66
sum of bucket length=66
[-0.06  0.06 -0.1 ]
[3.85803943 3.85803943 3.51484757]
[3, 45, 47]
[-0.16  0.16 -0.08]
[4.35954595 4.35954595 4.06923012]
[8, 42, 46]
[-0.46  0.46 -0.1

number of unique ID=49
sum of bucket length=49
[-0.32  0.32  0.46]
[6.48415433 6.48415433 4.07978247]
[16, 23, 34]
[-0.34  0.34 -0.28]
[5.09977169 5.09977169 2.96740573]
[17, 33, 36]
[-0.4   0.4  -0.44]
[4.56473111 4.56473111 4.36682565]
[20, 28, 30]
[-0.44  0.44  0.24]
[4.92510214 4.92510214 4.70116769]
[12, 22, 28]
[-0.32  0.32 -0.2 ]
[5.49781487 5.49781487 4.36525273]
[16, 34, 40]
[-0.38  0.38 -0.24]
[4.76270795 4.76270795 3.40082103]
[19, 31, 38]
[-0.44  0.44  0.34]
[4.42882399 4.42882399 3.90901986]
[17, 22, 28]
[ 0.12 -0.12 -0.36]
[9.4911522  9.4911522  8.54192809]
[6, 32, 44]
[-0.36  0.36 -0.42]
[4.70227969 4.70227969 3.71802211]
[18, 29, 32]
[-0.38  0.38  0.18]
[3.32109551 3.32109551 1.99522108]
[9, 19, 31]
[-0.38  0.38  0.44]
[4.02816138 4.02816138 3.71256507]
[19, 22, 31]
[-0.3   0.3  -0.46]
[2.74235167 2.74235167 2.72570083]
[15, 27, 35]
[-0.28  0.28 -0.36]
[4.73183447 4.73183447 4.4444585 ]
[14, 32, 36]
[-0.36  0.36  0.28]
[4.33506873 4.33506873 3.63950749]
[14, 18, 32]
[-0

...

[-0.26  0.26 -0.06]
[92.83666243 92.83666243 81.29205537]
[13, 37, 47]
[-0.4  0.4 -0.2]
[199.80773467 199.80773467 192.65944584]
[20, 30, 40]
[-0.5   0.34 -0.34]
[218.365      186.69344666 186.69344666]
[17, 25, 33]
[ 0.28 -0.28 -0.44]
[58.38659019 58.38659019 58.31461451]
[14, 28, 36]
[ 0.22 -0.22  0.48]
[74.84462942 74.84462942 69.3684742 ]
[11, 24, 39]
[-0.2  0.2 -0.3]
[10.04489384 10.04489384  9.86407257]
[10, 35, 40]
[-0.24  0.24 -0.44]
[37.86301388 37.86301388 36.93200979]
[12, 28, 38]
[ 0.24 -0.24  0.48]
[59.58353738 59.58353738 59.12256419]
[12, 24, 38]
[-0.06  0.06 -0.24]
[69.94301384 69.94301384 69.03847601]
[3, 38, 47]
[-0.12  0.12 -0.34]
[19.88544583 19.88544583 19.5935127 ]
[6, 33, 44]
[-0.5  -0.42  0.42]
[4.228      3.53021582 3.53021582]
[21, 25, 29]
[ 0.24 -0.24 -0.48]
[41.27772969 41.27772969 40.28693606]
[12, 26, 38]
[-0.22  0.22 -0.16]
[13.04640602 13.04640602 13.03899767]
[11, 39, 42]
[ 0.28 -0.28 -0.44]
[29.69614516 29.69614516 27.82685333]
[14, 28, 36]
[ 0.24 -0.2

[-0.04  0.04 -0.36]
[18.53896697 18.53896697 18.4429725 ]
[2, 32, 48]
[-0.42  0.42  0.06]
[15.67257122 15.67257122 15.40680659]
[3, 21, 29]
[-0.44  0.44  0.34]
[5.66312922 5.66312922 5.41723235]
[17, 22, 28]
[-0.14  0.14 -0.2 ]
[8.72202492 8.72202492 8.25624338]
[7, 40, 43]
[ 0.3 -0.3 -0.2]
[39.15759229 39.15759229 37.64185881]
[15, 35, 40]
[-0.36  0.36 -0.32]
[8.60888845 8.60888845 6.24715481]
[18, 32, 34]
[-0.38  0.38  0.42]
[11.52489102 11.52489102 11.35129544]
[19, 21, 31]
[-0.18  0.18 -0.36]
[46.86174984 46.86174984 45.8773701 ]
[9, 32, 41]
[ 0.32 -0.32 -0.04]
[80.38668457 80.38668457 78.53606737]
[16, 34, 48]
[-0.5  -0.18  0.18]
[65.223      60.36059765 60.36059765]
[9, 25, 41]
[-0.5   0.22 -0.22]
[11.546      10.36466192 10.36466192]
[11, 25, 39]
[-0.38  0.38  0.18]
[13.71789828 13.71789828 11.60159462]
[9, 19, 31]
[-0.12  0.12 -0.24]
[5.36185562 5.36185562 5.22154388]
[6, 38, 44]
[-0.14  0.14 -0.28]
[17.30160545 17.30160545 17.2983405 ]
[7, 36, 43]
[-0.16  0.16 -0.08]
[5.813932

[15, 29, 35]
[10, 40, 46]
[9, 41, 43]
[3, 42, 47]
[10, 40, 48]
[4, 42, 46]
[3, 11, 47]
[7, 14, 36]
[2, 44, 48]
[12, 35, 38]
[14, 34, 36]
[24, 26, 38]
[8, 13, 37]
[23, 27, 29]
[19, 31, 34]
[10, 30, 40]
[2, 8, 48]
[18, 32, 36]
[3, 38, 47]
[16, 19, 34]
[11, 37, 39]
[14, 36, 45]
[10, 25, 40]
[8, 34, 42]
[17, 29, 33]
[16, 34, 44]
[11, 36, 39]
[7, 25, 43]
[13, 25, 37]
[19, 31, 48]
[11, 24, 26]
[6, 17, 44]
[8, 23, 27]
[6, 28, 44]
[15, 22, 35]
[3, 43, 47]
[6, 28, 44]
[2, 8, 48]
[6, 25, 44]
[16, 34, 43]
[3, 25, 47]
[10, 25, 40]
[4, 21, 46]
Data_files/U21_LIHF_itd_file.csv
number of unique ID=115
sum of bucket length=109
[ 0.38 -0.38 -0.14]
[11.93310448 11.93310448 11.30576071]
[19, 31, 43]
[ 0.4 -0.4 -0.2]
[9.85237082 9.85237082 9.73921256]
[20, 30, 40]
[-0.26  0.26  0.48]
[162.12180334 162.12180334 161.9128212 ]
[13, 24, 37]
[-0.5  -0.18  0.18]
[19.199      18.57054356 18.57054356]
[9, 25, 41]
[-0.4   0.4   0.28]
[8.79251337 8.79251337 8.55036788]
[14, 20, 30]
[-0.06  0.06 -0.5 ]
[37.5725385 3

[-0.24  0.24  0.38]
[72.56229558 72.56229558 70.50789422]
[12, 19, 38]
[-0.5 -0.3  0.3]
[38.74       33.98412996 33.98412996]
[15, 25, 35]
[-0.22  0.22 -0.4 ]
[9.10099318 9.10099318 8.42848261]
[11, 30, 39]
[-0.16  0.16  0.32]
[80.84404426 80.84404426 78.16574942]
[8, 16, 42]
[-0.3   0.3  -0.38]
[121.18907801 121.18907801 119.17542298]
[15, 31, 35]
[-0.24  0.24  0.14]
[3.58128887 3.58128887 3.4248756 ]
[7, 12, 38]
[-0.5   0.46 -0.46]
[18.592      18.35985792 18.35985792]
[23, 25, 27]
[-0.34  0.34 -0.2 ]
[7.95563289 7.95563289 6.56292971]
[17, 33, 40]
[-0.04  0.04  0.38]
[51.63942518 51.63942518 49.39070074]
[2, 19, 48]
[-0.5  -0.16  0.16]
[125.224      113.10654295 113.10654295]
[8, 25, 42]
[-0.48  0.48 -0.22]
[5.9241791  5.9241791  5.13210115]
[24, 26, 39]
[ 0.48 -0.48 -0.44]
[76.86286212 76.86286212 76.11780418]
[24, 26, 28]
[-0.34  0.34 -0.2 ]
[54.227055   54.227055   52.72438633]
[17, 33, 40]
[ 0.16 -0.16 -0.4 ]
[6.09641627 6.09641627 6.09015902]
[8, 30, 42]
[-0.08  0.08  0.46]
[16

[7, 17, 43]
[15, 35, 40]
[11, 39, 42]
[18, 32, 40]
[18, 32, 38]
[9, 41, 45]
[14, 27, 36]
[8, 36, 42]
[7, 17, 33]
[6, 26, 44]
[7, 11, 43]
[2, 13, 48]
[20, 30, 40]
[13, 15, 35]
[19, 25, 31]
[15, 17, 33]
[22, 25, 28]
[12, 38, 48]
[24, 26, 47]
[14, 25, 36]
[14, 17, 36]
[4, 22, 28]
[21, 29, 41]
[9, 25, 41]
[16, 25, 34]
[22, 28, 35]
[9, 41, 44]
[15, 17, 33]
[5, 25, 45]
[18, 30, 32]
[11, 17, 39]
[18, 32, 41]
[2, 6, 48]
[17, 33, 45]
[10, 31, 40]
[12, 25, 38]
[13, 25, 37]
[2, 39, 48]
[19, 31, 41]
[22, 28, 39]
[6, 41, 44]
[13, 37, 41]
[9, 25, 41]
[12, 25, 38]
[12, 25, 38]
[21, 29, 38]
[23, 27, 40]
[11, 33, 39]
[13, 25, 37]
[21, 29, 46]
[3, 44, 47]
[10, 30, 40]
[10, 25, 40]
[11, 25, 39]
[5, 13, 37]
[10, 25, 40]
[17, 33, 45]
[9, 34, 41]
[18, 25, 32]
[11, 25, 39]
[16, 34, 43]
[16, 34, 48]
[15, 21, 29]
[8, 25, 42]
[11, 39, 43]
[2, 25, 48]
[13, 26, 37]
[22, 25, 28]
[13, 30, 37]
[12, 31, 38]
[11, 25, 39]
[9, 32, 41]
[16, 34, 42]
[3, 20, 30]
[19, 31, 35]
[12, 24, 38]
[17, 25, 33]
[14, 36, 43]
[12, 28, 